In [1]:
import pandas as pd
import os
from rapidfuzz import fuzz

In [2]:
# First let's get each player position and id for the season
def process_season_data(data_directory, seasons):
    for season in seasons:
        season_path = os.path.join(data_directory, season)

        if os.path.isdir(season_path):
            player_idlist_path = os.path.join(season_path, "player_idlist.csv")
            cleaned_players_path = os.path.join(season_path, "cleaned_players.csv")
            output_file_path = os.path.join(season_path, "processed_players.csv")

            if os.path.exists(player_idlist_path) and os.path.exists(cleaned_players_path):
                try:
                    player_idlist_df = pd.read_csv(player_idlist_path)
                    cleaned_players_df = pd.read_csv(cleaned_players_path)

                    merged_df = pd.merge(
                        player_idlist_df[['id', 'first_name', 'second_name']],
                        cleaned_players_df[['first_name', 'second_name', 'element_type']],
                        on=['first_name', 'second_name'],
                        how='inner'
                    )

                    # Rename element_type to position
                    merged_df.rename(columns={'element_type': 'position'}, inplace=True)

                    # Save the DataFrame to a new file
                    merged_df.to_csv(output_file_path, index=False)
                    print(f"Processed and saved: {output_file_path}")
                except Exception as e:
                    print(f"Error processing season {season}: {e}")
            else:
                print(f"Missing required files in {season_path}: player_idlist.csv or cleaned_players.csv")

In [3]:
data_directory = "Fantasy-Premier-League/data"
seasons = ["2024-25", "2023-24", "2022-23", "2021-22", "2020-21"]
process_season_data(data_directory, seasons)

Processed and saved: Fantasy-Premier-League/data/2024-25/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2023-24/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2022-23/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2021-22/processed_players.csv
Processed and saved: Fantasy-Premier-League/data/2020-21/processed_players.csv


In [4]:
def merge_player_ids(data_directory, seasons, output_file="master_player_list.csv"):
    player_data = {}
    next_unique_id = 1
    
    # We want to base ids on 2024-25 season
    main_season = "2024-25"
    main_season_path = os.path.join(data_directory, main_season, "processed_players.csv")
    
    if os.path.exists(main_season_path):
        try:
            main_processed_df = pd.read_csv(main_season_path)
            
            for _, row in main_processed_df.iterrows():
                full_name = f"{row['first_name']} {row['second_name']}"
                if full_name not in player_data:
                    player_data[full_name] = {
                        "First_Name": row['first_name'],
                        "Last_Name": row['second_name'],
                        "Unique_ID": row['id']
                    }
                # Add the 24_id for the main season
                player_data[full_name]['24_id'] = row['id']
                next_unique_id = max(next_unique_id, row['id'] + 1)  # Ensure greater ids to avoid reusing

        except Exception as e:
            print(f"Error processing {main_season_path}: {e}")
    else:
        print(f"Missing processed_players.csv for season: {main_season}")

    # Process other seasons
    for season in seasons:
        
        if season != main_season:
            season_path = os.path.join(data_directory, season, "processed_players.csv")
            if os.path.exists(season_path):
                try:
                    processed_df = pd.read_csv(season_path)
                    
                    season_short = season[:4][-2:]
                    
                    for _, row in processed_df.iterrows():
                        full_name = f"{row['first_name']} {row['second_name']}"
                        if full_name not in player_data:
                            player_data[full_name] = {
                                "First_Name": row['first_name'],
                                "Last_Name": row['second_name'],
                                "Unique_ID": next_unique_id
                            }
                            next_unique_id += 1
                        # Add the season ID
                        player_data[full_name][f"{season_short}_id"] = row['id']
                except Exception as e:
                    print(f"Error processing {season_path}: {e}")

    consolidated_df = pd.DataFrame.from_dict(player_data, orient='index').reset_index(drop=True)
    
    # Ensure all ID columns are integers
    id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
    consolidated_df[id_columns] = consolidated_df[id_columns].fillna(-1).astype(int)
    
    # Save the consolidated DataFrame to a CSV
    output_path = os.path.join(data_directory, output_file)
    consolidated_df.to_csv(output_path, index=False)
    print(f"Consolidated player data saved to {output_path}")

    return consolidated_df

In [5]:
merge_player_ids(data_directory, seasons)

Consolidated player data saved to Fantasy-Premier-League/data/master_player_list.csv


,First_Name,Last_Name,Unique_ID,24_id,23_id,22_id,21_id,20_id
0,Fábio,Ferreira Vieira,1,1,4,25,-1,-1
1,Gabriel,Fernando de Jesus,2,2,8,28,263,282
2,Gabriel,dos Santos Magalhães,3,3,5,16,-1,-1
3,Kai,Havertz,4,4,6,145,141,500
4,Karl,Hein,5,5,646,655,532,-1
...,...,...,...,...,...,...,...,...
1748,Theo,Corbeanu,1749,-1,-1,-1,-1,610
1749,Taylor,Perry,1750,-1,-1,-1,-1,611
1750,Lewis,Richards,1751,-1,-1,-1,-1,615
1751,Nigel,Lonwijk,1752,-1,-1,-1,-1,629


In [13]:
# Script below was used to get duplicate candidates

consolidated_file = "master_player_list.csv"
consolidated_path = os.path.join(data_directory, consolidated_file)
consolidated_df = pd.read_csv(consolidated_path)

SIMILARITY_THRESHOLD = 80
FIRST_NAME_SIMILARITY_THRESHOLD = 40

def is_pair(player_a, player_b, id_columns):
    # Check for ID overlaps across seasons
    id_overlap = any(
        player_a[col] != -1 and player_b[col] != -1
        for col in id_columns
    )
    if id_overlap:
        return False
    
    # Now we check the names similarity
    first_name_similarity = fuzz.partial_ratio(player_a["First_Name"], player_b["First_Name"])
    if first_name_similarity <= FIRST_NAME_SIMILARITY_THRESHOLD:
        return False
    
    # Check if one last name is contained in the other
    if player_a["Last_Name"] in player_b["Last_Name"] or player_b["Last_Name"] in player_a["Last_Name"]:
        return True

    # Calculate string similarity only if containment check fails
    similarity = fuzz.partial_ratio(player_a["Last_Name"], player_b["Last_Name"])
    return similarity >= SIMILARITY_THRESHOLD

# Identify all ID columns
id_columns = [col for col in consolidated_df.columns if col.endswith("_id")]
pairs = []
for i, player_a in consolidated_df.iterrows():
    for j, player_b in consolidated_df.iterrows():
        if i >= j:  # Avoid duplicate and self-pairs
            continue
        if is_pair(player_a, player_b, id_columns):
            pairs.append({
                "Player_A_First_Name": player_a["First_Name"],
                "Player_A_Last_Name": player_a["Last_Name"],
                "Player_A_Unique_ID": player_a["Unique_ID"],
                "Player_B_First_Name": player_b["First_Name"],
                "Player_B_Last_Name": player_b["Last_Name"],
                "Player_B_Unique_ID": player_b["Unique_ID"],
            })

# Convert pairs to a DataFrame
created_csv_dir = "created_csv"
pairs_df = pd.DataFrame(pairs)
if not pairs_df.empty:
    output_path = os.path.join(created_csv_dir, "players_with_potential_renames.csv")
    pairs_df.to_csv(output_path, index=False)
    print(f"Potential renamed players saved to {output_path}")
else:
    print("No potential renamed players found.")

Potential renamed players saved to created_csv/players_with_potential_renames.csv


In [7]:
# After manual checking the candidates now we have to
# - adjust their 

In [12]:
fuzz.partial_ratio("Edward", "Eddie")

57.14285714285714